# Benchmark latence/contexte

Objectif : Identifier les meilleurs modèles (qualité + vitess CPU) en testant l'impact de la taille du contexte sur le temps de réponse.

In [1]:
import subprocess
import time
from datetime import datetime
import json

## Fonctions :

In [7]:
def format_prompt(context: str, question: str):
    return f"""
Tu es un expert juridique en assurance. Réponds uniquement à la question, de manière claire, concise, sans emoji, sans HTML. Si tu ne sais pas, réponds "Je ne sais pas".

Contexte :
{context}

Question :
{question}
"""

In [8]:
def run_ollama(model: str, prompt: str):
    start = time.time()
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    end = time.time()
    output = result.stdout.decode("utf-8", errors="ignore")
    return {
        "time_total": round(end - start, 2),
        "response": output.strip()
    }

## Chargement des données

In [ ]:
"phi:2.7b-chat-v2-q4_K_M",
"dolphin3:8b-llama3.1-q4_K_M",
"gemma2:2b",
"gemma3n:e2b-it-q4_K_M",
"qwen3:1.7b-q4_K_M",

In [ ]:
TINY_MODELS = [
    "tinyllama:1.1b-chat-v0.6-q4_K_M",
    "llama3.2:1b-instruct-q4_K_M",
    "gemma3:1b-it-q4_K_M",
    "qwen3:0.6b-q4_K_M",
    "qwen3:1.7b-q4_K_M",
]

In [ ]:
COMPACT_MODELS = [
    "phi3:3.8b-mini-4k-instruct-q4_K_M",
    "gemma2:2b-instruct-q4_K_M",
    "gemma3:4b-it-q4_K_M",
    "deepseek-r1:1.5b-qwen-distill-q4_K_M"
]

In [ ]:
BIG_MODEL = [
    "mistral:7b-instruct-v0.3-q4_K_M",
    "llama3:8b-instruct-q4_K_M"
    ]

In [12]:
TEST_CASES = [
    {
        "no_context": "",
        "context_500": """Les biens assurés sont garantis pour les dommages causés directement par des événements énumérés aux articles suivants et dans les limites indiquées dans le tableau des montants maximums garantis. 
                            […]
                            Article 3 - Le vol et les actes de vandalisme. La mise en jeu de la garantie suppose que soit prouvée l'une des circonstances suivantes :
                            effraction des bâtiments assurés c'est-à-dire le forcement, la dégradation ou la destruction des dispositifs de fermeture ;
                            """,
        "context_1732": """Les biens assurés sont garantis pour les dommages causés directement par des événements énumérés aux articles suivants et dans les limites indiquées dans le tableau des montants maximums garantis. 
                            […]
                            Article 3 - Le vol et les actes de vandalisme. La mise en jeu de la garantie suppose que soit prouvée l'une des circonstances suivantes :
                            effraction des bâtiments assurés c'est-à-dire le forcement, la dégradation ou la destruction des dispositifs de fermeture ;
                            […]
                            L'inobservation des mesures de sécurité indiquées ci-dessous ayant permis ou facilité la réalisation du vol ou de l'acte de vandalisme entraîne une exclusion de garantie (sauf cas fortuit ou de force majeure).
                            - Entre 22 h 00 et 6 h 00, vous devez verrouiller vos portes, fermer vos fenêtres et autres ouvertures non munies de barreaux ou fermer leurs volets (ou persiennes).
                            - En cas d'absence, même de courte durée, vous devez fermer vos fenêtres et ouvertures, verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            - En cas d'inhabitation supérieure à 24 heures, vous devez utiliser tous les moyens de fermeture (volets, persiennes), verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            En cas d'inhabitation totale et continue de plus de 60 jours la garantie des objets précieux est suspendue et celle des autres biens mobiliers est limitée à 25 % du capital mobilier assuré.
                            """,
        "context_long_2845" : """Les biens assurés sont garantis pour les dommages causés directement par des événements énumérés aux articles suivants et dans les limites indiquées dans le tableau des montants maximums garantis. 
                            […]
                            Article 3 - Le vol et les actes de vandalisme. La mise en jeu de la garantie suppose que soit prouvée l'une des circonstances suivantes :
                            effraction des bâtiments assurés c'est-à-dire le forcement, la dégradation ou la destruction des dispositifs de fermeture ;
                            […]
                            L'inobservation des mesures de sécurité indiquées ci-dessous ayant permis ou facilité la réalisation du vol ou de l'acte de vandalisme entraîne une exclusion de garantie (sauf cas fortuit ou de force majeure).
                            - Entre 22 h 00 et 6 h 00, vous devez verrouiller vos portes, fermer vos fenêtres et autres ouvertures non munies de barreaux ou fermer leurs volets (ou persiennes).
                            - En cas d'absence, même de courte durée, vous devez fermer vos fenêtres et ouvertures, verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            - En cas d'inhabitation supérieure à 24 heures, vous devez utiliser tous les moyens de fermeture (volets, persiennes), verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            En cas d'inhabitation totale et continue de plus de 60 jours la garantie des objets précieux est suspendue et celle des autres biens mobiliers est limitée à 25 % du capital mobilier assuré.
                            […]
                            Dispositions particulières pour les dépendances et autres locaux assurés tels que caves et greniers, situés à la même adresse mais sans communication avec l'habitation :
                            Ce qui est garanti :
                            - le vol et les actes de vandalisme des biens mobiliers situés à l'intérieur de ces bâtiments sous condition qu'ils soient entièrement clos* et efficacement protégés (portes d'accès pleines avec verrous de sûreté, parties vitrées et autres ouvertures protégées par des volets ou barreaux);
                            - les frais de remplacement des serrures des portes privatives en cas de perte ou vol des clés en tout lieu ;
                            - la disparition ou détérioration des installations d'alarme affectées à la protection des biens assurés ;
                            - les détériorations des biens immobiliers assurés (ou la disparition de certains de leurs éléments) lors d'un vol, d'une tentative de vol ou d'un acte de vandalisme, y compris les inscriptions, tags et graffitis ;
                            """,
        "context_4260": """Les biens assurés sont garantis pour les dommages causés directement par des événements énumérés aux articles suivants et dans les limites indiquées dans le tableau des montants maximums garantis.
                            […]
                            Article 3 - Le vol et les actes de vandalisme. La mise en jeu de la garantie suppose que soit prouvée l'une des circonstances suivantes :
                            effraction des bâtiments assurés c'est-à-dire le forcement, la dégradation ou la destruction des dispositifs de fermeture ;
                            […]
                            L'inobservation des mesures de sécurité indiquées ci-dessous ayant permis ou facilité la réalisation du vol ou de l'acte de vandalisme entraîne une exclusion de garantie (sauf cas fortuit ou de force majeure).
                            - Entre 22 h 00 et 6 h 00, vous devez verrouiller vos portes, fermer vos fenêtres et autres ouvertures non munies de barreaux ou fermer leurs volets (ou persiennes).
                            - En cas d'absence, même de courte durée, vous devez fermer vos fenêtres et ouvertures, verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            - En cas d'inhabitation supérieure à 24 heures, vous devez utiliser tous les moyens de fermeture (volets, persiennes), verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            En cas d'inhabitation totale et continue de plus de 60 jours la garantie des objets précieux est suspendue et celle des autres biens mobiliers est limitée à 25 % du capital mobilier assuré.
                            […]
                            Dispositions particulières pour les dépendances et autres locaux assurés tels que caves et greniers, situés à la même adresse mais sans communication avec l'habitation :
                            Ce qui est exclu :
                            - le vol des objets précieux;
                            - les vols, actes de vandalisme et détériorations immobilières dans le locaux à usage collectif;
                            - le vol des vins et spiritueux;
                            Ce qui est garanti :
                            - le vol et les actes de vandalisme des biens mobiliers situés à l'intérieur de ces bâtiments sous condition qu'ils soient entièrement clos* et efficacement protégés (portes d'accès pleines avec verrous de sûreté, parties vitrées et autres ouvertures protégées par des volets ou barreaux);
                            - les frais de remplacement des serrures des portes privatives en cas de perte ou vol des clés en tout lieu ;
                            - la disparition ou détérioration des installations d'alarme affectées à la protection des biens assurés ;
                            - les détériorations des biens immobiliers assurés (ou la disparition de certains de leurs éléments) lors d'un vol, d'une tentative de vol ou d'un acte de vandalisme, y compris les inscriptions, tags et graffitis ;
                            […]
                            Les garanties optionnelles :
                            Les garanties énoncées ci-dessous vous sont accordées en supplément des autres garanties dans la mesure où vous les avez souscrites et dans les limites figurant dans le tableau des montants maximums garantis
                            Article 28 - Le matériel en tout lieu
                            Les appareils multimédia nomades
                            Ce qui est garanti :
                            - le vol de l'appareil multimédia nomade par effraction d'un local immobilier privatif clos et couvert;
                            En cas de vol garanti de votre téléphone mobile, vous devez, outre les indications d'ordre général (“La façon de procéder en cas de sinistre*”), faire la demande de mise hors service de la ligne concernée auprès de l'opérateur téléphonique dans les 2 jours ouvrés à partir du moment où vous en avez eu connaissance et nous fournir :
                            - en cas d'effraction, tout justificatif de l'effraction tel que facture de serrurier, de garagiste ;
                            """
    },
    {
        "question": "Mon vélo est-il couvert s'il est volé dans mon jardin ?",
        "context_court": """Les bicyclettes […] sont garanties au titre des biens mobiliers s'ils sont situés à l'intérieur de l'habitation ou des dépendances assurées.
            Ces biens peuvent être assurés en tout lieu, en cas de vol ou d'incendie, moyennant la souscription de la garantie optionnelle “Le matériel en tout lieu”.

            Les dommages causés par un événement garanti aux biens extérieurs situés sur le terrain de votre habitation (barbecues, portiques, pergolas, etc.) sont couverts si la garantie optionnelle « Biens extérieurs » est souscrite.""",
        "context_moyen": """""",
        "context_long" : """"""
    },
    {
        "question": "Y a-t-il des exclusions de garantie en cas de vol si j'ai laissé une fenêtre ouverte ?",
        "context_court": """L'inobservation des mesures de sécurité suivantes entraîne une exclusion de garantie (sauf cas fortuit ou force majeure) :
                            • entre 22 h et 6 h, vous devez verrouiller les portes, fermer les fenêtres et autres ouvertures ou fermer leurs volets ;
                            • en cas d'absence, même de courte durée, vous devez fermer les fenêtres, verrouiller les portes et activer le système de protection électronique dont vous disposez.""",
        "context_moyen": """""",
        "context_long" : """"""
    }
]

In [3]:
CONTEXT_RAG = {'context_rag': """\nExtrait du document 'CG_habitation_RP_protectrice' :\nélectroménagers endommagés.\nArticle 3 - Le vol et les actes de vandalisme\n  Dans quelles circonstances la garantie peut-elle être mise en jeu ?\nLa mise en jeu de la garantie suppose que soit prouvée l’une des circonstances suivantes :\n  effraction des bâtiments assurés c’est-à-dire le forcement, la dégradation ou la destruction des dispositifs de fermeture ;\n  pénétration dans les locaux assurés, à usage privatif, par escalade ou usage de fausses clés ;\n  introduction clandestine ou maintien à l’insu de l’assuré, dans les bâtiments assurés, alors que l’assuré lui-même ou \nune personne autorisée est présente dans les lieux ;\n  menaces ou violences sur l’assuré ou sur toute personne autorisée à être dans les locaux assurés ;\n  ruse c’est-à-dire l’utilisation d’une fausse qualité ou d’une fausse identité ayant permis l’introduction dans les lieux \net la réalisation du vol ;\n  vol commis pendant un incendie ;\n  vol “domestique” commis par les employés, en service, de l’assuré ou celui commis par des personnes invitées par \ncelui-ci (à condition qu’une plainte nominative soit déposée).\nPour les actes de vandalisme, inscriptions, tags et graffitis commis à l’extérieur sur les biens immobiliers assurés, ou \nle vol de certains de leurs éléments tels que volets, gouttières, tuiles, la garantie n’est pas subordonnée à l’une des \ncirconstances évoquées ci-dessus.\nExtrait du document 'CG_habitation_RP_protectrice' :\n18\n  Pour quels biens ou quels dommages, la garantie peut-elle jouer dans les circonstances évoquées ?\nCe qui est garanti :\nCe qui est exclu :\nOutre les exclusions communes à toutes les garanties et \ncelles relatives au non-respect des mesures de sécurité\n  les détériorations des biens immobiliers assurés (ou la \ndisparition de certains de leurs éléments) lors d’un vol, d’une \ntentative de vol ou d’un acte de vandalisme, y compris les \ninscriptions, tags et graffitis ;\n  les vols et actes de vandalisme commis dans des \nparties communes de l’immeuble ;\n  les vols et actes de vandalisme commis à l’aide de \nclés laissées à l’extérieur des locaux assurés : dans \nun boîtier sécurisé, dans la boîte aux lettres, sous \nle paillasson, dans le jardin, sur la terrasse, dans la \ncour ou sur la fenêtre ;\n  les vols et actes de vandalisme commis à l’aide \nde clés perdues ou dérobées sans que l’assuré n’ait \nprocédé au remplacement des serrures ou verrous \ncorrespondants (sauf cas de force majeure) ;\n  le vol des objets précieux dans les vérandas ;\n  le vol des animaux.\n  la disparition ou détérioration des installations d’alarme \naffectées à la protection des biens assurés ;\n  les frais de remplacement des serrures des portes \nprivatives des biens immobiliers assurés en cas de perte\nExtrait du document 'CG_habitation_RP_protectrice' :\n17\n  Exclusion en cas de non-respect des mesures de sécurité\nL’inobservation des mesures de sécurité indiquées ci-dessous ayant permis ou facilité la réalisation du vol ou \nde l’acte de vandalisme entraîne une exclusion de garantie (sauf cas fortuit ou de force majeure).\n  Entre 22 h 00 et 6 h 00, vous devez verrouiller vos portes, fermer vos fenêtres et autres ouvertures non munies \nde barreaux ou fermer leurs volets (ou persiennes).\n  En cas d’absence, même de courte durée, vous devez fermer vos fenêtres et ouvertures, verrouiller vos portes \net mettre en fonction tout système de protection électronique dont vous disposez.\n  En cas d’inhabitation supérieure à 24 heures, vous devez utiliser tous les moyens de fermeture (volets, \npersiennes), verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous \ndisposez.\n  En cas d’inhabitation\n  En cas d’inhabitation totale et continue de plus de 60 jours la garantie des objets précieux est suspendue \net celle des autres biens mobiliers est limitée à 25 % du capital mobilier assuré.\n  Les avantages de la protection électronique\nSi votre habitation est protégée par un système de détection d’intrusion en état de fonctionnement et activé, vous \nbénéficiez de la suppression de la franchise* contractuelle.\nSi ce système de détection d’intrusion est relié à une station de télésurveillance agréée APSAD dont les opérateurs \ncertifiés gèrent les alertes et organisent si nécessaire l'intervention d'un agent de sécurité, vous bénéficiez en plus :\n\\n
""",
"context_4260": """Les biens assurés sont garantis pour les dommages causés directement par des événements énumérés aux articles suivants et dans les limites indiquées dans le tableau des montants maximums garantis.
                            […]
                            Article 3 - Le vol et les actes de vandalisme. La mise en jeu de la garantie suppose que soit prouvée l'une des circonstances suivantes :
                            effraction des bâtiments assurés c'est-à-dire le forcement, la dégradation ou la destruction des dispositifs de fermeture ;
                            […]
                            L'inobservation des mesures de sécurité indiquées ci-dessous ayant permis ou facilité la réalisation du vol ou de l'acte de vandalisme entraîne une exclusion de garantie (sauf cas fortuit ou de force majeure).
                            - Entre 22 h 00 et 6 h 00, vous devez verrouiller vos portes, fermer vos fenêtres et autres ouvertures non munies de barreaux ou fermer leurs volets (ou persiennes).
                            - En cas d'absence, même de courte durée, vous devez fermer vos fenêtres et ouvertures, verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            - En cas d'inhabitation supérieure à 24 heures, vous devez utiliser tous les moyens de fermeture (volets, persiennes), verrouiller vos portes et mettre en fonction tout système de protection électronique dont vous disposez.
                            En cas d'inhabitation totale et continue de plus de 60 jours la garantie des objets précieux est suspendue et celle des autres biens mobiliers est limitée à 25 % du capital mobilier assuré.
                            […]
                            Dispositions particulières pour les dépendances et autres locaux assurés tels que caves et greniers, situés à la même adresse mais sans communication avec l'habitation :
                            Ce qui est exclu :
                            - le vol des objets précieux;
                            - les vols, actes de vandalisme et détériorations immobilières dans le locaux à usage collectif;
                            - le vol des vins et spiritueux;
                            Ce qui est garanti :
                            - le vol et les actes de vandalisme des biens mobiliers situés à l'intérieur de ces bâtiments sous condition qu'ils soient entièrement clos* et efficacement protégés (portes d'accès pleines avec verrous de sûreté, parties vitrées et autres ouvertures protégées par des volets ou barreaux);
                            - les frais de remplacement des serrures des portes privatives en cas de perte ou vol des clés en tout lieu ;
                            - la disparition ou détérioration des installations d'alarme affectées à la protection des biens assurés ;
                            - les détériorations des biens immobiliers assurés (ou la disparition de certains de leurs éléments) lors d'un vol, d'une tentative de vol ou d'un acte de vandalisme, y compris les inscriptions, tags et graffitis ;
                            […]
                            Les garanties optionnelles :
                            Les garanties énoncées ci-dessous vous sont accordées en supplément des autres garanties dans la mesure où vous les avez souscrites et dans les limites figurant dans le tableau des montants maximums garantis
                            Article 28 - Le matériel en tout lieu
                            Les appareils multimédia nomades
                            Ce qui est garanti :
                            - le vol de l'appareil multimédia nomade par effraction d'un local immobilier privatif clos et couvert;
                            En cas de vol garanti de votre téléphone mobile, vous devez, outre les indications d'ordre général (“La façon de procéder en cas de sinistre*”), faire la demande de mise hors service de la ligne concernée auprès de l'opérateur téléphonique dans les 2 jours ouvrés à partir du moment où vous en avez eu connaissance et nous fournir :
                            - en cas d'effraction, tout justificatif de l'effraction tel que facture de serrurier, de garagiste ;
                            """}


In [ ]:
from huggingface_hub import login
login(token="...")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it", token="...", trust_remote_code=True)


In [6]:
len(CONTEXT_RAG['context_rag'])

4366

In [9]:
tokens = tokenizer.encode(CONTEXT_RAG['context_rag'], truncation=False)
len(tokens)

1271

## Test

In [ ]:
QUESTION = "Suis-je couvert en cas d'effraction dans mon garage ?"

In [26]:
results = []
M = ["gemma3:4b-it-q4_K_M"]
for model in M :
    for ctx_label, ctx_text in CONTEXT_RAG.items():
        prompt = format_prompt(ctx_text, QUESTION)
        print(f"\n⏳ Testing model '{model}' with context '{ctx_label}'...")
        r = run_ollama(model, prompt)
        results.append({
            "timestamp": datetime.now().isoformat(),
            "model": model,
            "context_size": ctx_label,
            "response": r["response"],
            "time_total": r["time_total"],
            "response_length": len(r["response"]),
        })
        print(f"✅ {model} | {ctx_label} | {r['time_total']}s\nRéponse : {r['response'][:200]}...\n")

# Export results
with open("ollama_benchmark_results_rag.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\n📊 Résultats enregistrés dans 'ollama_benchmark_results_rag.json'")


⏳ Testing model 'gemma3:4b-it-q4_K_M' with context 'context_rag'...
✅ gemma3:4b-it-q4_K_M | context_rag | 95.8s
Réponse : Oui, vous êtes couvert en cas d'effraction dans votre garage, si l'effraction est prouvée (forcement, dégradation ou destruction des dispositifs de fermeture)....


⏳ Testing model 'gemma3:4b-it-q4_K_M' with context 'context_4260'...
✅ gemma3:4b-it-q4_K_M | context_4260 | 82.62s
Réponse : Oui, vous êtes couvert en cas d'effraction dans votre garage, sous certaines conditions. La garantie s'applique si le garage est un local immobilier privatif clos et efficacement protégé (portes d'acc...


📊 Résultats enregistrés dans 'ollama_benchmark_results_rag.json'
